In [1]:
# -*- coding: utf-8 -*-
# 📌 1️⃣ IMPORTS

from modules.Utils import read_decimal_file
from modules.Analysing import analyse_sequence , evaluate_generator
from modules.Generators import PythonGenerator, OurGenerator
from modules.Tests import Chi2Test, GapTest, MaximumTest, KSTest, PokerTest, CouponCollectorTest
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
# ===============================================
# 📌 2️⃣ PARAMÈTRES
# ===============================================
DECIMAL_FILE = "data/e2M.txt"
GRANULARITIES = [1000, 10000, 50000 , 100000]  # tailles des portions pour analyse locale
ALPHA = 0.05
TESTS = [Chi2Test(), GapTest(), MaximumTest(), KSTest(), PokerTest(), CouponCollectorTest()]

N_GEN = 100000  # taille d'une séquence générée
K_ITER = 100    # nombre de répétitions pour évaluer les générateurs



In [3]:
# =================================================
# 📌 3️⃣ LECTURE ET PRÉPARATION DES DÉCIMALES DE e
# =================================================
# => Récupération des 2M décimales en entiers (0-9)
decimals_e = read_decimal_file(DECIMAL_FILE)
print(f"Longueur totale des décimales de e : {len(decimals_e)}")


Longueur totale des décimales de e : 2000000


In [6]:
# ===============================================
# 📌 4️⃣ ANALYSE DE LA SÉQUENCE COMPLÈTE DES DÉCIMALES
# ===============================================
# => Séquence globale
global_res = analyse_sequence(
    sequence=decimals_e,
    tests=TESTS,
    alpha=ALPHA,
    granularities=None
)

# => Séquence par portions de granularités
granular_res = analyse_sequence(
    sequence=decimals_e,
    tests=TESTS,
    alpha=ALPHA,
    granularities=GRANULARITIES
)

print("\n✅ Analyse globale et par granularités des décimales de e terminée !")




✅ Analyse globale et par granularités des décimales de e terminée !


In [7]:
# ===============================================
# 📌 6️⃣ ÉVALUATION DES GÉNÉRATEURS
# ===============================================

# Création des générateurs
gen_python = PythonGenerator()
gen_our = OurGenerator(decimals=decimals_e)

# Évaluation Python Generator
df_python = evaluate_generator(gen_python,
                            granularities=GRANULARITIES,
                            seq_len=N_GEN,
                            n_repeat=K_ITER,
                            tests=TESTS,
                            alpha=ALPHA)
print("✅ Évaluation Python Generator terminée.")

# Évaluation Our Generator
df_our = evaluate_generator(gen_our,
                            granularities=GRANULARITIES,
                            seq_len=N_GEN,
                            n_repeat=K_ITER,
                            tests=TESTS,
                            alpha=ALPHA)
print("✅ Évaluation Our Generator terminée.")


✅ Évaluation Python Generator terminée.
✅ Évaluation Our Generator terminée.


In [ ]:

# Fusion des résultats
df_comparison = pd.concat([df_python.hist_df, df_our.hist_df], ignore_index=True)

# Sauvegarde
df_comparison.to_csv("comparison_results.csv", index=False)

print("\n Résultats sauvegardés dans 'comparison_results.csv'")



📁 Résultats sauvegardés dans 'comparison_results.csv'


In [ ]:
# ===============================================
# 📌 7️⃣ VISUALISATION : Taux de rejet par test et par générateur
# ===============================================
summary = df_comparison.groupby(["Generator", "Test"])["Reject_H0"].mean().reset_index()
summary["Rejection_Rate (%)"] = summary["Reject_H0"] * 100

plt.figure(figsize=(12, 6))
sns.barplot(data=summary, x="Test", y="Rejection_Rate (%)", hue="Generator")
plt.title("📊 Taux de rejet H0 par test statistique et par générateur")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n✅ Pipeline complet terminé ! Tu peux inclure le CSV et le graphique dans ton rapport.")
